In [117]:
import requests, boto3, pandas as pd, os, sys, pprint
from pathlib import Path
pp = pprint.PrettyPrinter(indent=2, compact=True, width=80)

#TODO: Change path accordingly in handler
sys.path.append('../')

In [118]:
# These imports will change accordingly
from images.entities.DailyUpload import DailyUpload
from images.lib import db_helpers

In [119]:
ddb = boto3.client("dynamodb", region_name="ap-south-1")

# Replace placeholder_value with value from the event object.

# headers = {"User-Agent": "placeholder_valueAPI/0.0.1"}

# Initializing without the actual username and password
# data = {"grant_type": "password", "username": "", "password": ""}


REDDIT_AUTH_URL = 'https://www.reddit.com/api/v1/access_token'
REDDIT_ACCOUNTS_TABLE_NAME = 'RedditAccountsTable-dev'

subreddit='funny'

daily_upload = DailyUpload(subreddit=subreddit)

In [125]:
class RedditAccount:
    def __init__(self,subreddit, ddb):
        self.subreddit= subreddit
        self.client_id = None
        self.secret_key = None
        self.username = None
        self.password = None
        self.auth = None
        self.headers = {"User-Agent": f"{subreddit}API/0.0.1"}
        self.data = {"grant_type": "password", "username": None, "password": None}
        self.access_token = None
        self.ddb = ddb
        
    def key(self):
        return {
            "PK" : { "S": self.subreddit }
        }
                
    def fetch_and_update_account_details(self, REDDIT_ACCOUNTS_TABLE_NAME):
        try:
            response = self.ddb.get_item(TableName=REDDIT_ACCOUNTS_TABLE_NAME, Key=self.key())   
            item = RedditAccount.deserialize_item(response['Item'])
            self.client_id = item['personal_use_script']
            self.secret_key = item['secret_key']
            self.username = item['username']
            self.password = item['password']

        except Exception as e:
            print(f"Failed with exception: {e.args[0]}")
            
        self.data['username'] = self.username
        self.data['password'] = self.password
            
    @staticmethod
    def deserialize_item(item):
        new_item = {}
        for key in item:
            new_item[key] = RedditAccount.extract_value(item[key])

        return new_item
    
    @staticmethod
    def extract_value(dictionary):    
        data_type, value = list(dictionary.keys())[0], list(dictionary.values())[0]

        if data_type == "S":
            return value
                
    def authenticate_with_api(self):
        self.auth = requests.auth.HTTPBasicAuth(self.client_id, self.secret_key)
                
    def fetch_access_token(self, REDDIT_AUTH_URL):
        # Authorise and request for access token from Reddit API
        res = requests.post(REDDIT_AUTH_URL, auth=self.auth, data=self.data, headers=self.headers)
        self.access_token = res.json()['access_token']
        self.headers['Authorization'] = f"bearer {self.access_token}"
                
    def fetch_posts(self, url,  params={}):
        res = requests.get(url, headers=self.headers, params=params)
        return res.json()
                
                

## Get values from reddit accounts table

In [126]:
reddit_account = RedditAccount(subreddit=subreddit, ddb=ddb)
reddit_account.fetch_and_update_account_details(REDDIT_ACCOUNTS_TABLE_NAME)

# try:    
#     response = ddb.get_item(TableName=REDDIT_ACCOUNTS_TABLE_NAME, Key=get_item)
#     item = db_helpers.deserialize_from_db_item(response["Item"])
#     CLIENT_ID = item["personal_use_script"]
#     SECRET_KEY = item["secret_key"]
#     USERNAME = item["username"]
#     PASSWORD = item["password"]

# except Exception as e:
#     print(f"Failed with exception: {e.args[0]}")

## Authorise with reddit

In [127]:
# auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

# data["username"] = USERNAME
# data["password"] = PASSWORD

# # Request for access token from Reddit API
# res = requests.post(REDDIT_AUTH_URL, auth=auth, data=data, headers=headers)

# ACCESS_TOKEN = res.json()["access_token"]

# headers["Authorization"] = f"bearer {ACCESS_TOKEN}"

reddit_account.authenticate_with_api()
reddit_account.fetch_access_token(REDDIT_AUTH_URL)


## Send request for posts (with params, limit 100 posts)

In [128]:
REDDIT_API_URL_TOP = "https://oauth.reddit.com/r/placeholder_value/top"
REDDIT_API_URL_TOP = REDDIT_API_URL_TOP.replace("placeholder_value", subreddit)

# res = requests.get(REDDIT_API_URL_TOP, headers=headers, params={'limit': '100'})
posts = reddit_account.fetch_posts(REDDIT_API_URL_TOP, params = {'limit': '100'})


## Build table of urls

In [130]:
df_top = pd.DataFrame()
total_duration = 0

for post in posts["data"]["children"]:
    if post["data"]["is_video"]:
        df_top = df_top.append(
            {
                "title": post["data"]["title"],
                "upvote_ratio": post["data"]["upvote_ratio"],
                "ups": post["data"]["ups"],
                "downs": post["data"]["downs"],
                "score": post["data"]["score"],
                "url": post["data"]["url"],
            },
            ignore_index=True,
        )

        total_duration += int(post["data"]["media"]["reddit_video"]["duration"])

df_top = df_top.sort_values(
    ["score", "upvote_ratio", "ups"], ascending=False, axis=0
)



In [136]:
df_top

,title,upvote_ratio,ups,downs,score,url
0,"Oh, so you can just walk up.",0.93,49604.0,0.0,49604.0,https://v.redd.it/bs62f4e9dcg71
1,dick falls out while running,0.87,30813.0,0.0,30813.0,https://v.redd.it/6m0v1c4wubg71
2,He's nervous,0.97,3313.0,0.0,3313.0,https://v.redd.it/pelrc774tfg71
4,How to start bad day,0.96,1277.0,0.0,1277.0,https://v.redd.it/abm6uan7sfg71
3,Indian dudes chilling in car - Shadup,0.92,1087.0,0.0,1087.0,https://v.redd.it/sgwl3juyvag71
5,Penguin Shenanigans,0.96,877.0,0.0,877.0,https://v.redd.it/4vbqxydrkfg71
6,Work It,0.89,440.0,0.0,440.0,https://v.redd.it/mtgcitvv4eg71
7,Every time,0.89,248.0,0.0,248.0,https://v.redd.it/iwxsu94qcfg71
8,Okay this has a smidgen of NSFW.,0.65,178.0,0.0,178.0,https://v.redd.it/ihnn3miogbg71
11,"My new kitten Auggie, he is not the brightest :)",0.95,98.0,0.0,98.0,https://v.redd.it/nvycsrze4gg71


In [131]:
daily_upload.urls =  daily_upload.urls + df_top['url'].tolist()
daily_upload.total_duration = total_duration
daily_uploads_table = "DailyUploadsTable-dev"

In [132]:
daily_upload.urls

['https://v.redd.it/bs62f4e9dcg71',
 'https://v.redd.it/6m0v1c4wubg71',
 'https://v.redd.it/pelrc774tfg71',
 'https://v.redd.it/abm6uan7sfg71',
 'https://v.redd.it/sgwl3juyvag71',
 'https://v.redd.it/4vbqxydrkfg71',
 'https://v.redd.it/mtgcitvv4eg71',
 'https://v.redd.it/iwxsu94qcfg71',
 'https://v.redd.it/ihnn3miogbg71',
 'https://v.redd.it/nvycsrze4gg71',
 'https://v.redd.it/or8btijdgfg71',
 'https://v.redd.it/nbx2pcttbcg71',
 'https://v.redd.it/81jgsyigncg71',
 'https://v.redd.it/qq63tdjk2cg71',
 'https://v.redd.it/t32m9g60dcg71',
 'https://v.redd.it/v5vund8ycdg71']

In [133]:
daily_upload.serialize_date_subreddit()

{'PK': {'S': '2021-08-10'},
 'SK': {'S': 'funny'},
 'urls': {'L': [{'S': 'https://v.redd.it/bs62f4e9dcg71'},
   {'S': 'https://v.redd.it/6m0v1c4wubg71'},
   {'S': 'https://v.redd.it/pelrc774tfg71'},
   {'S': 'https://v.redd.it/abm6uan7sfg71'},
   {'S': 'https://v.redd.it/sgwl3juyvag71'},
   {'S': 'https://v.redd.it/4vbqxydrkfg71'},
   {'S': 'https://v.redd.it/mtgcitvv4eg71'},
   {'S': 'https://v.redd.it/iwxsu94qcfg71'},
   {'S': 'https://v.redd.it/ihnn3miogbg71'},
   {'S': 'https://v.redd.it/nvycsrze4gg71'},
   {'S': 'https://v.redd.it/or8btijdgfg71'},
   {'S': 'https://v.redd.it/nbx2pcttbcg71'},
   {'S': 'https://v.redd.it/81jgsyigncg71'},
   {'S': 'https://v.redd.it/qq63tdjk2cg71'},
   {'S': 'https://v.redd.it/t32m9g60dcg71'},
   {'S': 'https://v.redd.it/v5vund8ycdg71'}]}}

In [134]:
res = ddb.transact_write_items(
    TransactItems=[
        {
            "Put": {
                "TableName": daily_uploads_table,
                "Item": daily_upload.serialize_date_subreddit()
            }
        },        
        {
            "Put": {
                "TableName": daily_uploads_table,
                "Item": daily_upload.serialize_subreddit_date()
            }
        }
            
    ]
)